In [ ]:
#import the required libraries
import tweepy
from twython import Twython
import pandas as pd
from time import sleep


#Assemble your tweety API info here
access_token = OAUTH_TOKEN = "Yours"
access_token_secret = OAUTH_TOKEN_SECRET = "Yours"
consumer_key = APP_KEY = "Yours"
consumer_secret = APP_SECRET = "Yours"


#Connect to twitter/make an engine
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

class PyFeshTweetry:        
    def _is_response(self, idx):
        '''
        Details: See if tweet is a response
        Params - idx: id of the tweet (not id_str)
        '''
        try:
            tweet = twitter.show_status(id=idx)
        except Exception as e:
            return False, None, 'Error'
        if tweet['in_reply_to_status_id']:
            print("Got a reply.")
            r = tweet['in_reply_to_status_id']
            return True, tweet, r
        print(f"Tweeet with id {idx} is not a reply.")
        return False, None, None

    def asili(self, idx):
        '''
        Details: Find and assemble into a pandas dataframe the conversation.
        Params - idx: id of the tweet (not id_str)
        '''
        statusId = idx
        tweets = []
        created = []
        screenName = []
        idRepliedTo = []

        rp = True
        while rp:
            status, tweet, r = self._is_response(statusId)
            if r == 'Error':
                rp = False
                print(f'Tweet with id {statusId} does not exist!')
                return None
            if status:
                tweets.append(tweet['text'])
                created.append(tweet['created_at'])
                screenName.append(tweet['user']['screen_name'])
                idRepliedTo.append(r)
                statusId = r
            else:
                rp = False
                print(f'End of the Show!')

        if len(tweets) == len(created) == len(screenName) == len(idRepliedTo):
            df = pd.DataFrame()
            df['tweets'] = tweets
            df['created'] = created
            df['in_reply_to_id'] = idRepliedTo
            df['in_reply_to_sname'] = screenName

        return df
        
        
#Scrapping funciton
def profile_Scrapper(candidate):
    """
    Log into twitter and scrape all tweets by referenced user grabbing the indicated fields
    Make info in a dataframe and save to csv file
    """
    
    #Auth into Tiwtter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #Interface/login
    api = tweepy.API(auth, wait_on_rate_limit = True)
    
    #list of variables we are interested in in the returned json
    created_at = []
    status_id = []
    status_id_str = []
    text= []
    source = []
    in_reply_to_screen_name = []
    contributors = []
    quoted_status_id = []
    quote_count = []
    reply_count = []
    retweet_count = []
    favorite_count = []
    in_reply_to_status_id_str = []

    #List of headings needed to build a dataframe of the pulled fields
    count = 0 #variable to hold count of tweets pulled and enable sleep periods

    print("Scrapping tweets...")
    #Commence scrape
    for status in tweepy.Cursor(api.user_timeline, screen_name=candidate, tweet_mode="extended").items():
        #Rest at various intervals to ensure good dev behavior and reduce chances of timeouts
        if count == 500 or count == 900 or count == 1300 or count == 2300:
            print("We have {} tweets. Taking a 5 seconds nap".format(count))
            sleep(5)
            
        try:
            try:
                created_at.append(status.created_at)
            except:
                created_at.append("Not Availabe")
            status_id.append(status.id)
            status_id_str.append(status.id_str)
            in_reply_to_status_id_str.append(status.in_reply_to_status_id_str)
            in_reply_to_screen_name.append(status.in_reply_to_screen_name)
            text.append(status.full_text)
            source.append(status.source)
            try:
                contributors.append(status.contributors)
            except:
                contributors.append("Not Availabe")
            try:
                quoted_status_id.append(status.quoted_status_id)
            except:
                quoted_status_id.append("Not Availabe")
            try:
                quote_count.append(status.quote_count)
            except:
                quote_count.append("Not Availabe")
            try:
                reply_count.append(status.reply_count)
            except:
                reply_count.append("Not Availabe")
            try:
                retweet_count.append(status.retweet_count)
            except:
                retweet_count.append("Not Availabe")
            try:
                favorite_count.append(status.favorite_count)
            except:
                favorite_count.append("Not Availabe")
            print("status " + str(count))
            count+=1
        except:
            print("Tweep Error")
            sleep(30)


    print("Building dataframe...")
    Tweets = pd.DataFrame()
    Tweets["created_at"] = created_at
    Tweets["status_id"] = status_id
    Tweets["status_id_str"] = status_id_str
    Tweets["text"] = text
    Tweets['in_reply_to_status_id_str'] = in_reply_to_status_id_str
    Tweets["source"] = source
    Tweets["in_reply_to_screen_name"] = in_reply_to_screen_name
    Tweets["contributors"] = contributors
    Tweets["quoted_status_id"] = quoted_status_id
    Tweets["quote_count"] = quote_count
    Tweets["reply_count"] = reply_count
    Tweets["retweet_count"] = retweet_count
    Tweets["favorite_count"] = favorite_count
    
    #Save to csv file
    Tweets.to_csv(candidate + '.csv', index=False)
    
    print("Done!")
    return Tweets

In [ ]:
go = PyFeshTweetry()

In [ ]:
result = go.asili(1234567890)

In [ ]:
result